<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#탐색적-분석" data-toc-modified-id="탐색적-분석-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>탐색적 분석</a></span><ul class="toc-item"><li><span><a href="#결측치-확인-및-처리" data-toc-modified-id="결측치-확인-및-처리-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>결측치 확인 및 처리</a></span></li></ul></li></ul></div>

Kaggle 사이트 : https://www.kaggle.com/c/zillow-prize-1

현재 참고 커널 : https://www.kaggle.com/viveksrinivasan/zillow-eda-on-missing-values-multicollinearity

In [ ]:
# !pip install missingno
import pylab
import calendar
import missingno as msno
from datetime import datetime

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from scipy import stats
from scipy.stats import kendalltau

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

### 탐색적 분석

In [ ]:
train = pd.read_csv('train_2016_v2.csv', parse_dates=["transactiondate"])
properties = pd.read_csv('properties_2016.csv')

In [ ]:
print(train.head(2))
print(f'train.shape :{ train.shape}')

In [ ]:
properties.head(2)

In [ ]:
print(f'properties.shape :{ properties.shape}')

In [ ]:
# 2016 데이터와 
merged = pd.merge(train,properties,on="parcelid",how="left")

In [ ]:
merged.shape

In [ ]:
merged.info()

In [ ]:
merged.head(3).transpose()

In [ ]:
# 변수별 null값 시각화
null_value = merged.isnull().sum().reset_index()
null_value.columns = ['feature','CountNA']
null_value = null_value.sort_values('CountNA', ascending=False)
fig, ax = plt.subplots()
fig.set_size_inches(11, 15)
graph = sns.barplot(x='CountNA',y='feature',data=null_value, ax=ax, color="#34495e")
graph.set(xlabel='Count of NA values', ylabel='Features', title = 'NA value counts')
ax.plot([merged.shape[0]/2, merged.shape[0]/2], [0,null_value.shape[0]], "r--")
plt.show()

#### 결측치 확인 및 처리

- 결측치가 0.5이상인 컬럼은 삭제하고 모델링한다. (결측치가 0.5이상인 변수를 모델링에 포함할 시 더 많은 정보손실이 있을것이라 판단)
- logerror를 제외한 30개 변수가 있음

In [ ]:
# null 값 50%이상 제외
upper_50 = (merged.isnull().sum()/merged.shape[0] > 0.5)
merged_50 = merged.columns[upper_50]
new_merged = merged.drop(merged_50,axis=1)
new_merged.shape

In [ ]:
null_value = new_merged.isnull().sum().reset_index()
null_value.columns = ['feature','CountNA']
null_value = null_value.sort_values('CountNA', ascending=False)
fig, ax = plt.subplots()
fig.set_size_inches(11, 15)
graph = sns.barplot(x='CountNA',y='feature',data=null_value, ax=ax, color="#34495e")
graph.set(xlabel='Count of NA values', ylabel='Features', title = 'NA value counts', )
ax.plot([merged.shape[0]/2, merged.shape[0]/2], [0,null_value.shape[0]], "r--")
plt.show()

In [ ]:
new_merged.info()

In [ ]:
new_merged.corr().style.background_gradient(cmap='coolwarm')